In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
files = sorted(glob('output-tamil/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

12954

In [4]:
import mp
import copy

minimum_score = 6

def loop(files):
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            results.append(data[i])

    return results

In [5]:
results = mp.multiprocessing(files, loop, cores = 50)

100%|██████████| 259/259 [00:28<00:00,  9.00it/s]

100%|██████████| 259/259 [00:28<00:00,  9.00it/s]

100%|██████████| 259/259 [00:28<00:00,  8.99it/s]



100%|██████████| 259/259 [00:28<00:00,  8.95it/s]


 98%|█████████▊| 255/259 [00:28<00:00,  8.64it/s]

 99%|█████████▉| 256/259 [00:28<00:00,  9.08it/s]

100%|██████████| 259/259 [00:29<00:00,  8.92it/s]

 99%|█████████▉| 257/259 [00:29<00:00,  9.92it/s]

100%|██████████| 259/259 [00:29<00:00,  8.78it/s]

100%|██████████| 259/259 [00:29<00:00,  8.77it/s]

100%|██████████| 4/4 [00:02<00:00,  1.72it/s]


In [6]:
len(results)

544068

In [7]:
results[0]

{'predict_ta': [50258,
  50287,
  50360,
  50365,
  13535,
  4516,
  6980,
  5036,
  5511,
  12008,
  18169,
  4234,
  2133,
  4234,
  3923,
  36001,
  4742,
  2133,
  4742,
  3923,
  18198,
  19433,
  6467,
  8402,
  8491,
  4516,
  6993,
  2133,
  12008,
  5036,
  4742,
  3923,
  4234,
  5036,
  8012,
  2133,
  8012,
  3923,
  12776,
  6980,
  2133,
  16793,
  17167,
  2133,
  15094,
  2133,
  23656,
  6993,
  4234,
  4742,
  2133,
  4742,
  6467,
  4742,
  2133,
  4742,
  4516,
  6993,
  2133,
  23656,
  10078,
  2133,
  10078,
  12008,
  18169,
  4742,
  3923,
  6980,
  2133,
  12776,
  8708,
  8094,
  2133,
  8094,
  8402,
  16504,
  10478,
  6993,
  3923,
  6980,
  2133,
  16504,
  10478,
  6980,
  2133,
  35032,
  3923,
  4742,
  2133,
  4742,
  13535,
  10478,
  15297,
  2133,
  6421,
  4516,
  8491,
  4742,
  3923,
  12776,
  8708,
  16793,
  4516,
  8491,
  6980,
  2133,
  12894,
  5511,
  2133,
  8094,
  3923,
  13786,
  3923,
  8094,
  4516,
  8012,
  2133,
  8012,
  4516,


In [8]:
with open('prepared-tamil-original.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 544068/544068 [00:21<00:00, 25525.61it/s]


In [22]:
from huggingface_hub import HfApi
api = HfApi()

In [10]:
api.upload_file(
    path_or_fileobj='prepared-tamil-original.jsonl',
    path_in_repo='prepared-tamil-original.jsonl',
    repo_id='huseinzol05/pseudolabel-tamil-large-v3-timestamp',
    repo_type='dataset',
)

prepared-tamil-original.jsonl:   0%|          | 0.00/1.67G [00:00<?, ?B/s]

'https://huggingface.co/datasets/huseinzol05/pseudolabel-tamil-large-v3-timestamp/blob/main/prepared-tamil-original.jsonl'

In [11]:
len(files)

12954

In [16]:
import mp
import copy
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'

minimum_score = 6

def loop(files):
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            a = tokenizer._decode_asr(
                [{'tokens': np.array([data[i]['predict_ta']])}], 
                return_timestamps = True, return_language = 'ta', 
                time_precision = 0.02)[1]['chunks']
            
            a = [a_['text'] for a_ in a]

            results.extend(a)

    return results

In [1]:
results = mp.multiprocessing(files, loop, cores = 50)

In [18]:
len(results)

768049

In [19]:
results[:10]

[' விமான போக்குவரத்து தொரையில் பாதுகாப்பு அம்சங்கள் உலகத்தரத்தில் உள்ள போதும் அவற்றை மேலும் மேம்படுத்த வேண்டியது அவசியம் இன்று குறிப்பிட்டார்.',
 ' பிலாய் எக்கு தொழிச்சாலையில் நேரிட்ட தீவிபத்தில் வீரழந்தோரின் கொடுப்பத்தினருக்கு தலா 30 லட்சும் ரூபாய் நிவாரண உதவி வழங்கப்படும் என்ற மத்தி அமைச்சத்திர விரைந்தர் சிங் குரிவுள்ளார்.',
 ' விபத்தை நிகழ்ந்த பகுதியை நேற்று பார்வையிட்டாவர் விபத்தில் உயிரடந்தோரின் குடும்பத்தினர் ஒருவருக்கு எக்கு தொழிச்சாலையில் வேலைவாய்ப்பு அழிக்கவிடும் என்றும் அதிகாரிகளுக்கு அறிவுரத்தினார்.',
 ' பலத்த காயமடைந்தவர்களுக்கு 15 லேச்சும் ரூபாயும்',
 ' லேசான காயமடைந்தவர்களுக்கு 2 லேச்சும் ரூபாயும்',
 ' நிவாரணத்தோகி வளங்கப்படும் என்றும் அமிச்சர் அருவிதார்.',
 ' இந்த தொழிச்சாலையில் நேற்ற முன் தினம் நேரிடத்தி விபத்தில் 12 பேரு உயிரழந்தனர்',
 ' பத்து பேர் காயபடைந்தடர்',
 ' விமான எரிப்பொருளுக்கான கலால் வரியை 14 சதவிதத்திலிருந்து 11 சதவிதமாக மத்தி அரசு குறைத்துள்ளது.',
 ' விமான சரக்கு போக்கு வருத்து கொள்கையி மத்தியாரசு விரைவில் வெளியிட்டு நடை முழிப்படுத்த உள்ளதாக, அத்துரைக்கான 

In [20]:
with open('tamil-text.texts', 'w') as fopen:
    for t in tqdm(set(results)):
        if not len(t):
            continue
        
        fopen.write(f'{json.dumps(t)}\n')

100%|██████████| 676530/676530 [00:01<00:00, 408611.17it/s]


In [21]:
!ls -lh tamil-text.texts

-rw-r--r-- 1 ubuntu ubuntu 314M Apr 13 04:34 tamil-text.texts


In [23]:
api.upload_file(
    path_or_fileobj='tamil-text.texts',
    path_in_repo='tamil-text.texts',
    repo_id='mesolitica/pseudolabel-tamil-large-v3-timestamp',
    repo_type='dataset',
)

tamil-text.texts:   0%|          | 0.00/329M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-tamil-large-v3-timestamp/blob/main/tamil-text.texts'